### Qgis, centerline add length, convert to point ,add index , export as csv
```
Algorithm 'Field calculator' starting…
Input parameters:
{ 'FIELD_LENGTH' : 10, 'FIELD_NAME' : 'length', 'FIELD_PRECISION' : 3, 'FIELD_TYPE' : 0, 'FORMULA' : '$length', 'INPUT' : 'G:/810_change_field/linestrings_single_clip.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length.shp' }

Algorithm 'Extract vertices' starting…
Input parameters:
{ 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice.shp' }

Algorithm 'Field calculator' starting…
Input parameters:
{ 'FIELD_LENGTH' : 10, 'FIELD_NAME' : 'index', 'FIELD_PRECISION' : 3, 'FIELD_TYPE' : 1, 'FORMULA' : '@row_number', 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice_index.shp' }

```


### etree csv to xml

In [1]:
import csv
import xml.etree.ElementTree as ET

In [2]:
root = ET.Element ('network')
root.set('version','1.0')
nodes=ET.SubElement(root, 'nodes')
links=ET.SubElement(root, 'links')

In [3]:
with open ("centerline_vertices.csv",'rt',encoding='utf-8') as f:
    current_group = None
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT, FID,length, vertex_ind, vertex_par, vertex_p_1, distance, angle, index = row
        
        #single_node
        ET.SubElement(nodes, 'node',{'nodeId': index, 'geometry':WKT[7:-1], 'linkId':FID, 'vertex_ind':vertex_ind})
        #links
        if current_group is None or FID != current_group.get('linkId'):
            current_group = ET.SubElement(links, 'link',{'linkId':FID,'length':length })
        
        
        if (vertex_ind=='0'):
            current_link = ET.SubElement(current_group, 'nodes',{'nodeId':index})   
        else: 
            list_nodeId = current_link.get('nodeId')
            current_link.set('nodeId',list_nodeId+" "+index)

In [4]:
for link in links.findall('link'):
    #set node1 node2 of link
    list_node = link.find('nodes').get('nodeId').split(' ')
    link.set('node1', list_node[0])
    link.set('node2',list_node[-1])

In [5]:
tree = ET.ElementTree(root)
ET.indent(tree)
tree.write('output.xml',encoding='utf-8')